In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_info_columns', 10000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)


In [2]:

def get_dataframe_summary(df):
    """
    Returns a summary DataFrame for the given DataFrame.
    
    The summary includes:
      - Data Type
      - Non Null Count
      - Null Count
      - Null Percentage
      - Unique Values count
    """
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    
    summary_df = pd.DataFrame({
        'Data Type': df.dtypes,
        'Non Null Count': df.count(),
        'Null Count': df.isna().sum(),
        'Null Percentage': (df.isna().sum() / len(df) * 100).round(2),
        'Unique Values': [df[col].nunique() for col in df.columns],
    })
    
    return summary_df

In [3]:
from pathlib import Path
import sys

# 1) Where is this notebook?
notebook_dir = Path.cwd()

# 2) Climb up until you get to the folder that contains "app/"
#    parents[2] goes up from objetivo_2 → notebooks → objetivos → …
#    count how many levels from objetivo_2 to BOTS_RPA: in your case it's 8 levels
project_root = notebook_dir.parents[8]

# 3) Insert it at front of sys.path
sys.path.insert(0, str(project_root))

# 4) Now imports of "app.…" will succeed




In [4]:
BASE_DIR = Path.cwd().parent.parent.parent.parent.parent.parent.parent.parent.parent
SAVE_DIR_EXTRACT_EXCEL = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "excel"/ "CORTE 4.xlsx" 
SAVE_DIR_EXTRACT_SGA_335 = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sga_335" / "335_24_20250624_012937.xlsx"
CID_CUISMP_PATH = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sharepoint_cid_cuismp" / "MINPU - CID-CUISMP - AB (27).xlsx"
DIR_PARADAS_RELOJ = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "pausa_cliente" / "330_24_20250624_012937.xlsx"
DIR_WORD_DATOS = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "word_datos" / "COMPONENTE 2-DATOS.docx"
DIR_WORD_TELEFONIA = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "word_telefonia" / "COMPONENTE 4 - TELEFONOS.docx"

In [5]:
from app.modules.sga.minpub.report_validator.service.objetivos.etl.extract.corte_excel import extract_corte_excel
from app.modules.sga.minpub.report_validator.service.objetivos.etl.extract.informe_tecnico import (
    extract_tecnico_reports_without_hours_last_dates
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.informe_tecnico import ( 
    preprocess_df_word_informe_tecnico
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.informe_tecnico.datos import ( 
merge_word_datos_informe_corte_excel
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.informe_tecnico.telefonia import ( 
merge_word_telefonia_informe_corte_excel
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.corte_excel import ( 
    preprocess_corte_excel
)


In [6]:
df_corte_excel = extract_corte_excel(SAVE_DIR_EXTRACT_EXCEL, skipfooter=0)
df_word_datos_informe_tec =  extract_tecnico_reports_without_hours_last_dates(DIR_WORD_DATOS)
df_word_telefonia_informe_tec = extract_tecnico_reports_without_hours_last_dates(DIR_WORD_TELEFONIA)

In [7]:
df_corte_excel = preprocess_corte_excel(df_corte_excel)
df_word_datos_informe_tec =  preprocess_df_word_informe_tecnico(df_word_datos_informe_tec)
df_word_telefonia_informe_tec = preprocess_df_word_informe_tecnico(df_word_telefonia_informe_tec)

In [8]:

df_corte_excel = preprocess_corte_excel(df_corte_excel)
df_word_datos_informe_tec =  preprocess_df_word_informe_tecnico(df_word_datos_informe_tec)
df_word_telefonia_informe_tec = preprocess_df_word_informe_tecnico(df_word_telefonia_informe_tec)


In [9]:
    #INFORME TECNICO - DATOS - EXCEL
df_matched_word_datos_informe_tecnico_corte_excel = merge_word_datos_informe_corte_excel(
        df_word_datos_informe_tec,
        df_corte_excel,
        'both'
        )
    
    #INFORME TECNICO - TELEFONIA - EXCEL
df_matched_word_telefonia_informe_tecnico_corte_excel = merge_word_telefonia_informe_corte_excel(
        df_word_telefonia_informe_tec,
        df_corte_excel,
        'both'
        )

In [ ]:

from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_2.o2_informe_tecnico_validator import validate_informe_tecnico_word


df_vali_tel = validate_informe_tecnico_word(df_matched_word_telefonia_informe_tecnico_corte_excel, 'COMPONENTE IV' )
row = df_vali_tel[df_vali_tel['nro_incidencia'] == '21832090']
row

In [ ]:
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_2.o2_informe_tecnico_validator import build_failure_messages_validate_informe_tecnico_word


df_mess = build_failure_messages_validate_informe_tecnico_word(df_vali_tel)
#df_mess_TELE = build_failure_messages_validate_informe_tecnico_word(df_vali_tel)
df_mess

In [ ]:
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_2.o2_informe_tecnico_validator import build_failure_messages_validate_informe_tecnico_word


df_mess = build_failure_messages_validate_informe_tecnico_word(df_vali_tel)
#df_mess_TELE = build_failure_messages_validate_informe_tecnico_word(df_vali_tel)
df_mess

In [ ]:
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_2.o2_informe_tecnico_validator import build_failure_messages_validate_informe_tecnico_word


df_mess = build_failure_messages_validate_informe_tecnico_word(df_vali_tel)
#df_mess_TELE = build_failure_messages_validate_informe_tecnico_word(df_vali_tel)
df_mess

In [ ]:
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_2.o2_informe_tecnico_validator import build_failure_messages_validate_informe_tecnico_word


df_mess = build_failure_messages_validate_informe_tecnico_word(df_vali_tel)
#df_mess_TELE = build_failure_messages_validate_informe_tecnico_word(df_vali_tel)
df_mess

In [ ]:
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_2.o2_informe_tecnico_validator import build_failure_messages_validate_informe_tecnico_word


df_mess = build_failure_messages_validate_informe_tecnico_word(df_vali_tel)
#df_mess_TELE = build_failure_messages_validate_informe_tecnico_word(df_vali_tel)
df_mess

In [ ]:
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_2.o2_informe_tecnico_validator import build_failure_messages_validate_informe_tecnico_word


df_mess = build_failure_messages_validate_informe_tecnico_word(df_vali_tel)
#df_mess_TELE = build_failure_messages_validate_informe_tecnico_word(df_vali_tel)
df_mess